In [4]:
import tensorflow as tf
from tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

#因所用tensorflow版本为2.x，tensorflow.examples.tutorials.mnist已被移除，故采用以下命令来配置环境
# conda config --add channels https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/free/
# conda install tensorflow==1.14.0

#调整参数：学习率调小，正确率无提高；批次100->200，正确率无提高；weight初始化stddev（标准差）0.1->0.01以下,显著提高，0.1效果不稳定，有时候acc一直卡在0.085

learning_rate = 1e-4
keep_prob_rate = 0.7 # 
max_epoch = 2000
def compute_accuracy(v_xs, v_ys):
    global prediction
    y_pre = sess.run(prediction, feed_dict={xs: v_xs, keep_prob: 1})
    correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = sess.run(accuracy, feed_dict={xs: v_xs, ys: v_ys, keep_prob: 1})
    return result

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.005)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    # 每一维度  滑动步长全部是 1， padding 方式 选择 same
    # 提示 使用函数  tf.nn.conv2d

    strides = [1, 1, 1, 1]  # 在每个维度上的滑动步长，这里都设置为1表示在每个位置都进行卷积
    padding = 'SAME'  # 使用 "same" padding 方式，保持输入和输出的尺寸一致

    # 执行卷积操作
    conv = tf.nn.conv2d(x, W, strides=strides, padding=padding)

    return conv

def max_pool_2x2(x):
    # 滑动步长 是 2步; 池化窗口的尺度 高和宽度都是2; padding 方式 请选择 same
    # 提示 使用函数  tf.nn.max_pool
    
    ksize = [1, 2, 2, 1]  # 池化窗口的尺寸，这里设置为 2x2
    strides = [1, 2, 2, 1]  # 在每个维度上的滑动步长，这里设置为 2x2
    padding = 'SAME'  # 使用 "same" padding 方式，保持输入和输出的尺寸一致

    # 执行池化操作
    pool = tf.nn.max_pool(x, ksize=ksize, strides=strides, padding=padding)

    return pool

# define placeholder for inputs to network
xs = tf.placeholder(tf.float32, [None, 784])/255.
ys = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)
x_image = tf.reshape(xs, [-1, 28, 28, 1])

#  卷积层 1
## conv1 layer ##

W_conv1 = tf.Variable(tf.random_normal([7, 7, 1, 32]))  # patch 7x7, in size 1, out size 32
b_conv1 = tf.Variable(tf.random_normal([32]))                     
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1) # 卷积  自己选择 选择激活函数
h_pool1 = max_pool_2x2(h_conv1) # 池化               

# 卷积层 2
W_conv2 = tf.Variable(tf.random_normal([5, 5, 32, 64])) # patch 5x5, in size 32, out size 64
b_conv2 = tf.Variable(tf.random_normal([64]))
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2) # 卷积  自己选择 选择激活函数
h_pool2 = max_pool_2x2(h_conv2) # 池化

#  全连接层 1
## fc1 layer ##
W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# 全连接层 2
## fc2 layer ##
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
prediction = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)


# 交叉熵函数
cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction),
                                              reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    for i in range(max_epoch):
        batch_xs, batch_ys = mnist.train.next_batch(100)
        sess.run(train_step, feed_dict={xs: batch_xs, ys: batch_ys, keep_prob:keep_prob_rate})
        if i % 100 == 0:
            print(compute_accuracy(
                mnist.test.images[:1000], mnist.test.labels[:1000]))


Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
0.18
0.933
0.954
0.968
0.97
0.973
0.98
0.98
0.973
0.979
0.982
0.976
0.98
0.983
0.978
0.978
0.974
0.977
0.982
0.986
